In [1]:
# This script aggregates all the reviews for restaurants.

In [2]:
import json
import numpy as np
from rake_nltk import Rake
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import re

In [45]:
restaurants = []
with open ('/Users/jiamingqu/Desktop/yelp_dataset/restaurant.txt', 'r') as f:
    for line in f.readlines():
        restaurants.append(line.strip())

print(len(restaurants))

59371


In [4]:
restaurants_reviews = {}
count = 0

with open ('/Users/jiamingqu/Desktop/yelp_dataset/review.json', 'r') as f:
    for line in f.readlines():
        review = json.loads(line)
        business_id = review["business_id"]
        if business_id in restaurants:
            restaurants_reviews.setdefault(business_id, []).append(review)
        else:
            continue
            
        count += 1
        if count % 50000 == 0:
            print(count)

f.close()

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000


In [5]:
# average amount of reviews
review_count = np.array([])
for k,v in restaurants_reviews.items():
    review_count = np.append(review_count, len(v))

In [6]:
np.mean(review_count)

70.76997187178925

In [18]:
# use RAKE to extract keywords

In [9]:
# first aggregate reviews
restaurants_all_reviews = {}
review_word_count = np.array([])
count = 0

for k,v in restaurants_reviews.items():
    
    all_reviews_list = []
    for i in v:
        all_reviews_list.append(i["text"])
    all_reviews = " ".join(all_reviews_list)
    
    review_words = len(all_reviews.split(" "))
    review_word_count = np.append(review_word_count, review_words)
    
    restaurants_all_reviews[k] = all_reviews

In [10]:
# average count of words in reviews
np.mean(review_word_count)

7710.874972629735

In [13]:
def clean_text(text):

    '''
    Args: a list of documents
    Returns: a list of cleaned text
    '''

    # clean and tokenize document string
    raw_text = text.lower()
    raw_text = re.sub(r"[^\w\s]", "", raw_text)
    tokens = nltk.word_tokenize(raw_text)

    # remove stop words from tokens
    stop = stopwords.words("english")
    cleaned_tokens = [token for token in tokens if token not in stop]

    return " ".join(cleaned_tokens)

In [14]:
restaurant_all_reviews_keywords = {}

for k, v in restaurants_all_reviews.items():
    
    r = Rake()
    r.extract_keywords_from_text(v)
    ranked_list = r.get_ranked_phrases()
    
    keywords = " ".join(ranked_list[:100])
    
    restaurant_all_reviews_keywords[k] = clean_text(keywords)

In [55]:
keyword_count = np.array([])

for v in restaurant_all_reviews_keywords.values():
    keyword_count = np.append(keyword_count, len(v.split(" ")))

# average count of words in keywords
np.mean(keyword_count)

317.67438648498427

In [46]:
# sanity check
assert len(restaurant_all_reviews_keywords.items()) == len(restaurants)

In [47]:
with open ("/Users/jiamingqu/Desktop/yelp_dataset/restaurant.reviews/restaurant.keywords.json", 'w') as f:
    json.dump(restaurant_all_reviews_keywords, f)